In [18]:
import tensorflow as tf
import tensorflow_transform as tft
import json
import pandas as pd
import numpy as np
from utils.third_pass_utils import *

In [17]:
# data paths
end_date = "2023-10-18"
data_filename_suffix = "part-00000-of-00881"
data_filename = f"gs://etldata-prod-search-ranking-data-hkwv8r/feature_logging_training_data_purchase_with_predicted_score/query_pipeline_web_organic/{end_date}/results/{data_filename_suffix}"

# model & tft paths
model_root = "gs://training-dev-search-data-jtzn/neural_ranking/third_pass/third-pass-perso-add-noscore-web-18/model_root"
tft_output_path = f"{model_root}/preprocess_fit/{end_date}"

# generated data paths
examples_output_dir = "gs://training-dev-search-data-jtzn/neural_ranking/third_pass/temp/1698974462187/908724743918/third-pass-perso-add-noscore-web-18-development-1698974462187/beam-generate_-4350830870467182592/examples_output_dir"
gen_example_suffix = "train/part-08694-of-08695.tfrecord"
example_data_tfr_filename = f"{examples_output_dir}/{gen_example_suffix}"

In [3]:
# load only one line
with tf.io.gfile.GFile(data_filename) as f:
    for line in f:
        grouped_docs = json.loads(line)
        break

In [4]:
tft_output = tft.TFTransformOutput(tft_output_path)
tft_layer = tft_output.transform_features_layer()
raw_feature_spec = tft_output.raw_feature_spec()
transformed_feature_spec = tft_output.transformed_feature_spec()

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


In [5]:
transformed_data = process(grouped_docs, raw_feature_spec, tft_layer)

In [7]:
len(transformed_data[0].keys())

126

In [11]:
transformed_data[0].keys()

dict_keys(['candidateInfo.contextualInfo[name=user].shopId_recentlyViewedShopIds50FV1#shopId:overlap', 'candidateInfo.docInfo.listingInfo.listingSearchVisit14d.numCarts_log1p_standardized', 'candidateInfo.contextualInfo[name=user].shopId_recentlyViewedListingShopIds50FV1#shopId:overlap', 'candidateInfo.docInfo.listingInfo.activeListingBasics.totalOrders_log1p_standardized', 'contextualInfo[name=browser].candidateInfo.recentlyCartaddedListingIds50FV1#listingId_listingIdCartsVPCG#keys:overlap_log1p_standardized', 'sample_weight', 'candidateInfo.contextualInfo[name=user].listingId_recentlyCartaddedListingIds50FV1#listingId:overlap', 'contextualInfo[name=user].candidateInfo.recentlyPurchasedListingIds50FV1#listingId_listingIdCartsVPCG#keys:overlap_log1p_standardized', 'candidateInfo.candidateInfo.favoriteCountPT24hFV1_purchaseCountPT168hFV1:ratio_log1p_standardized', 'candidateInfo.candidateInfo.usShipCost_priceUsd:ratio_nan_log1p_standardized', 'contextualInfo[name=user].candidateInfo.rec

In [9]:
transformed_data[0]['sample_weight']

<tf.Tensor: shape=(34,), dtype=float32, numpy=
array([1.      , 1.      , 1.      , 1.      , 1.      , 1.      ,
       1.      , 1.      , 1.      , 1.      , 1.      , 1.      ,
       1.      , 1.      , 1.      , 1.      , 1.      , 1.      ,
       1.      , 1.      , 1.      , 1.      , 1.      , 1.      ,
       1.      , 1.      , 3.252265, 1.      , 1.      , 1.      ,
       1.      , 1.      , 1.      , 1.      ], dtype=float32)>

In [12]:
transformed_data[0]['attributions_score']

<tf.Tensor: shape=(34,), dtype=float32, numpy=
array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       50.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)>

In [13]:
len(transformed_feature_spec)

126

In [20]:
example_data = tf.data.TFRecordDataset(example_data_tfr_filename)

In [22]:
transformed_feature_spec.pop("attributions_score")

FixedLenFeature(shape=[], dtype=tf.float32, default_value=None)

In [23]:
df_list = []
for record in example_data:
    result = tf.io.parse_single_example(record, transformed_feature_spec)
    curr_res = {
        "sample_weight": result['sample_weight'].numpy(),
#         "attributions_score": result['attributions_score'].numpy(),
    }
    df_list.append(curr_res)

2023-11-06 10:03:36.973388: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: candidateInfo.candidateInfo.avgPropensity_numCarts:prod_nan_log1p_standardized (data type: float) is required but could not be found.


InvalidArgumentError: {{function_node __wrapped__ParseExampleV2_Tdense_125_num_sparse_0_ragged_split_types_0_ragged_value_types_0_sparse_types_0_device_/job:localhost/replica:0/task:0/device:CPU:0}} Feature: candidateInfo.candidateInfo.avgPropensity_numCarts:prod_nan_log1p_standardized (data type: float) is required but could not be found. [Op:ParseExampleV2]

In [ ]:
# df_tfrecord = pd.json_normalize(df_list)